# Coding Question

## CTC Loss

In [ ]:
import numpy as np

In [ ]:
def ctc_loss(log_probs):
    """
    Compute CTC loss based on the log-probability matrix.
    """

    final_prob =-np.log(log_probs)
    loss = np.sum(final_prob)

    return loss

In [ ]:
log_probs = np.array([
    [0.6, 0.2, 0.1, 0.1],
    [0.1, 0.7, 0.1, 0.1],
    [0.1, 0.1, 0.7, 0.1],
    [0.1, 0.6, 0.1, 0.2],
])

loss = ctc_loss(log_probs)
print(loss)

27.9797278902181


## Greedy Decoding
**You are NOT allowed to use np.argmax or any similar function**

In [160]:
def greedy_ctc_decode(probs, idx2char):
    """
    Greedy CTC decoding:
      - probs: list of lists, shape (T, V)
      - idx2char: dict mapping index -> character
    """
    T,C = probs.shape
    # print(T,C)
    list_greedy = []

    # 1) Argmax over classes at each timestep
    for t in range(T):
      idx = 0
      max_val = probs[t][0]
      for char in range(C):
          val = probs[t][char]
          if max_val< val:
            max_val = val
            idx = char
      list_greedy.append(idx)
      # print(list_greedy)
    #2) Collapse repeats and remove blanks
    final_list = []
    char_list= []
    prev = None

    # 3) Map indices to characters
    # 4) Join characters and return the predicted word

    for char in (list_greedy):
      if list_greedy[char]!=prev and list_greedy[char]!=0:
        prev = list_greedy[char]
        # print(prev)
      final_list.append(prev)

      char_list.append(idx2char[prev])
    # 4) Join characters and return the predicted word
    # idx2char[prev]
    return char_list,final_list

In [161]:
    # Vocabulary:
    # 0 = blank '_'
    # 1 = 'A'
    # 2 = 'B'
    # 3 = 'C'
    idx2char = {
        0: "_",
        1: "A",
        2: "B",
        3: "C"
    }

    # probs[t][v] = probability of symbol v at time t
    probs =np.array( [
        [0.1, 0.6, 0.2, 0.1],  # t0 -> 'A'
        [0.1, 0.7, 0.1, 0.1],  # t1 -> 'A'
        [0.5, 0.3, 0.1, 0.1],  # t2 -> blank
        [0.1, 0.1, 0.7, 0.1],  # t3 -> 'B'
        [0.1, 0.1, 0.7, 0.1],  # t4 -> 'B'
        [0.2, 0.1, 0.1, 0.6],  # t5 -> 'C'
    ])

    decoded = greedy_ctc_decode(probs, idx2char)
    print("Decoded word:", decoded)

Decoded word: (['A', 'A', 'A', 'A', 'A', 'B'], [1, 1, 1, 1, 1, 2])


# Discusison Quesiton

1.because we only need to calculate the sum of the negative log over the probabilities matrix

---

2. We calculate all possible probabilities over for the sentence (phonemes) so that we could find the best alignment per the audio for each word.
We need to add a blank to cover 3 cases
- char->blank  = char (stay)
- char->char = char (skip)
- char1->char2 = char1 - char2 (move)

we need a decoder beside adding blanks
*Ex:*
- Greedy -->calculate max prob over each timestep
- Beam-->calculate max prob per k of sequence




---

3. Beam has a better stategy than Greedy because it calculates the probability per k sequence (ex: k = 3) which are the best sequences so far per the timestep

- while the algorithm is working, if the one of the sequence return a bad max prob , the algo search for another best sequence (alternative).
- this ensures that we care about the accumlative sequence per all timestep (beam) rather that taking the max per each timestep (greedy)

